In [ ]:
#For this project we need two library
#1. opencv to read images those we have
# 2. easy ocr to extract text from images

In [ ]:
#If you have been working with OpenCV for some time, you should have noticed 
#that in most scenarios OpenCV utilizes CPU, which doesn’t always guarantee you the desired performance.
#To tackle this problem,in 2010 a new module that provides GPU acceleration using CUDA was added to OpenCV.
#INSHORT YOU CAN SAY THAT IT USED FOR PARALLEL COMPUTING
#or we say use power of grafix of cuda

In [ ]:
!nvcc --version


In [ ]:
# we have cuda_version 11.2 so we have to download from pytorch.org..

In [ ]:
!pip install easyocr
# it pretrained model
!pip install imutils
#it is image processing functions such as translation, rotation, resizing, skeletonization,
# displaying Matplotlib images, sorting contours, detecting edges
!pip install opencv-python-headless==4.1.2.30
#if u not write headless it will be not downloaded in google colab
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117

In [ ]:
# import all necessaries libraries

In [ ]:
import cv2
from matplotlib import pyplot as plt
import imutils
import numpy as np
import easyocr


In [ ]:
#image preprocessing(means converting colored image to gray image

In [ ]:
img=cv2.imread("/content/drive/MyDrive/car_imageeeeeeeee.jpg")
gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#cv2.cvtColor is for convertor
plt.imshow(cv2.cvtColor(gray,cv2.COLOR_BGR2RGB))
# for matplotlib expect RGB formate

In [ ]:
# Apply filtter and edge detection
bfilter=cv2.bilateralFilter(gray,11,11,17) # to reduce noise by bilateralfotler method
edged=cv2.Canny(bfilter,30,200) # for edge detection by Canny algorithm
plt.imshow(cv2.cvtColor(edged,cv2.COLOR_BAYER_BG2RGB))
# again for matplotlib expect RGB formate



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# find contour and and apply mask
## Contours can be explained simply as a curve joining all the continuous points
## mask to hide
keypoints=cv2.findContours(edged.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
# we know that edged is a numpy array and to copy array we use .copy().it should be binary image
# here two important parameter mode,method
#The mode cv2.RETR_TREE finds all the promising contour lines and reconstructs a full hierarchy of nested contours.
# The method cv2.CHAIN_APPROX_SIMPLE returns only the endpoints that are necessary for drawing the contour line
#like aqure and also avoid memory problem
contours=imutils.grab_contours(keypoints)
contours=sorted(contours,key=cv2.contourArea,reverse=True)[:10]
# here we are sorting all contours in decending order

In [ ]:
location=None
for contour in contours:
  apporx=cv2.approxPolyDP(contour,10,True) # 
  if len(apporx)==4:
    location=apporx
    break

In [ ]:
location

In [ ]:
# now we have to mask unnecessary area
mask=np.zeros(gray.shape,np.uint8)
new_image=cv2.drawContours(mask,[location],0,255,-1)
new_image=cv2.bitwise_and(img,img,mask=mask)

In [ ]:
plt.imshow(cv2.cvtColor(new_image,cv2.COLOR_BGR2RGB))

In [ ]:
(x,y)=np.where(mask==255)
(x1,y1)=(np.min(x),np.min(y))
(x2,y2)=(np.max(x),np.max(y))
croped_image=gray[x1:x2+1,y1:y2+1]# 1 for to give little bit buffer


In [ ]:
plt.imshow(cv2.cvtColor(croped_image,cv2.COLOR_BGR2RGB))

In [ ]:
# use easy_ocr to read text
reader=easyocr.Reader(['en'])  # for english language
result=reader.readtext(croped_image)
result

In [ ]:
# plot the result
text=result[0][-2]
font=cv2.FONT_HERSHEY_SIMPLEX
res = cv2.putText(img, text=text, org=(apporx[0][0][0],apporx[1][0][1]+60), fontface=font,
fontScale=1, color=(0,255,0), thickness=2,lineType= cv2.LINE_AA)
res = cv2.rectangle(img, tuple(approx[0][0]), tuple(approx[2][0]), (0,255,0),3)
plt.imshow(cv2.cvtColor(res,cv2.COLOR_BGR2RGB))
